**Dependencies:**

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq

  Using cached crewai-0.28.8-py3-none-any.whl.metadata (13 kB)
  Using cached crewai_tools-0.1.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached langchain_community-0.0.29-py3-none-any.whl.metadata (8.3 kB)
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached embedchain-0.1.123-py3-none-any.whl.metadata (9.3 kB)
  Using cached instructor-0.5.2-py3-none-any.whl.metadata (10 kB)
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached openai-1.52.0-py3-none-any.whl.metadata (24 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.27.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached regex-2023.12.25-cp310-cp310-manylinux_2_17_x86_64.manylin

**Libereies:**

In [5]:
# Warning control
import warnings
from crewai import Agent, Task, Crew,Process
import os
from google.colab import userdata
from langchain_groq import ChatGroq
warnings.filterwarnings('ignore')
from crewai_tools import SerperDevTool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from crewai_tools import BaseTool
import json
import requests
from jinja2 import Template


In [81]:

class GroqChatLLM(ChatGroq):
    def __init__(self, temperature=0, model_name="llama-3.2-90b-text-preview", api_key=None):
        api_key = userdata.get("GROQ_API_KEY")

        if not api_key:
            raise ValueError("GROQ_API_KEY must be provided either as an argument or in the environment variables.")

        # Directly call the superclass (ChatGroq) initialization
        super().__init__(
            temperature=temperature,
            model_name=model_name,
            api_key=api_key
        )
######################################################################################################################
# from langchain_ollama import ChatOllama
# class OllamaLLM(ChatOllama):
    # def __init__(self, temperature=0, model_name="llama3.1"):
    #     # Directly call the superclass (ChatOllama) initialization with the fixed base URL
    #     super().__init__(
    #         temperature=temperature,
    #         model=model_name,
        # )

######################################################################################################################


In [82]:
llm=GroqChatLLM()

In [8]:
llm.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_time': 0.032010838, 'prompt_time': 0.009680737, 'queue_time': 0.006509873000000001, 'total_time': 0.041691575}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-b88ed19a-ced5-4c73-a22c-873466d2794d-0')

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [9]:
# Set up the Serper API key
os.environ["SERPER_API_KEY"] = "633c9f4f684e57cb300edf5a39a465f85f800c13"

# Create a Serper search tool
search_tool = SerperDevTool()

In [10]:

class CurrencyConverterTool(BaseTool):
    name: str = "Currency Converter Tool"
    description: str = "Converts an amount from one currency to another using real-time exchange rates."

    def _run(self, amount: str, from_currency: str, to_currency: str) -> str:
        """
        Convert an amount from one currency to another using real-time exchange rates.
        :param amount: The amount to convert in the source currency.
        :param from_currency: The currency to convert from.
        :param to_currency: The currency to convert to.
        :return: A string with the converted amount or an error message.
        """
        try:
            #api_key = os.environ.get('EXCHRATE_API_KEY')
            api_key = "a71dffbb1968f78f3cf3e22f" # will expire on 11th Oct 2024

            if not api_key:
                return "Error: API key not found. Set 'EXCHRATE_API_KEY' in environment variables."

            amount = float(amount)  # Convert amount to float

            # API URL with from_currency and to_currency
            url = f"https://v6.exchangerate-api.com/v6/{api_key}/pair/{from_currency}/{to_currency}"
            response = requests.get(url)
            response.raise_for_status()  # Raise exception for HTTP errors
            data = response.json()

            # Check for a successful response
            if data.get("result") != "success":
                return f"Error: Failed to retrieve exchange rate. {data.get('error-type', 'Unknown error')}"

            # Access the conversion rate
            rate = data['conversion_rate']
            converted = amount * rate

            return f"{converted:.2f}"

        except ValueError:
            return "Error: Invalid amount. Please provide a numeric value."
        except requests.RequestException as e:
            return f"Error fetching exchange rates: {str(e)}"

class CalculateTool(BaseTool):
    name: str = "Calculate Tool"   #tool name
    description: str = "Evaluates a mathematical expression and returns the result as a float."

    def _run(self, expression: str) -> float:
        """
        Evaluate a mathematical expression and return the result as a float.
        :param expression: A string representing the mathematical expression to evaluate.
        :return: The result of the evaluation as a float. If an error occurs during evaluation, returns NaN.
        """
        try:
            print('-> TOOL-CALCULATE CALLED')
            return float(eval(expression))
        except Exception as e:
            return float('nan')


In [11]:
calculate_tool=CalculateTool()
currency_converter_tool=CurrencyConverterTool()


In [12]:
tools = [calculate_tool,currency_converter_tool]

In [13]:
for i in range(len(tools)):
    print(tools[i])
    print("#"*50)

name='Calculate Tool' description="Calculate Tool(expression: 'string') - Evaluates a mathematical expression and returns the result as a float." args_schema=<class 'abc.CalculateToolSchema'> description_updated=False cache_function=<function BaseTool.<lambda> at 0x797c91785510>
##################################################
name='Currency Converter Tool' description="Currency Converter Tool(amount: 'string', from_currency: 'string', to_currency: 'string') - Converts an amount from one currency to another using real-time exchange rates." args_schema=<class 'abc.CurrencyConverterToolSchema'> description_updated=False cache_function=<function BaseTool.<lambda> at 0x797c91785510>
##################################################


In [83]:
# Defining the Jinja2 template for the system prompt
system_prompt_template = """

Question: {{ query }}.

{% if cot_example %}
Chain of Thought Example(s):
    {{ cot_example }}
{% endif %}

Begin!
"""

In [84]:


# Assuming you have a researcher persona and system prompt template
calculator_persona = {
    "role": "mathematical assistant",
    "goal": "help users evaluate mathematical expressions accurately",
    "backstory": """You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information
     allows you to provide actionable insights, identify future trends, and assist researchers and professionals in
     navigating the rapidly evolving landscape of technology."""
}
calculator_agent= Agent(
    llm=llm,
    tools=[calculate_tool],  # Ensure tools are passed correctly
    role=calculator_persona["role"],
    goal=calculator_persona["goal"],  # Pass an actual topic
    backstory=calculator_persona["backstory"]  # Add the missing backstory
    , allow_delegation=False, allow_self_execution=True
)

# Example COT for the calculate task
calculate_cot_example = """
Question: What is the result of 23 * 7 + 15?
Thought: I need to perform a mathematical calculation. I'll use the Calculate Tool for this.
Action: Calculate Tool: (expression: 23 * 7 + 15)
PAUSE
Observation: The result is 176.0
Thought: The calculation is complete. I can now provide the answer.
Answer: The result of 23 * 7 + 15 is 176.
"""

# Define the task using the Task object
calculate_task = Task(
    description=Template(system_prompt_template).render(
query="{query}",
       cot_example=calculate_cot_example
    ),
    agent=calculator_agent,
    expected_output="""Answer the mathematical expression provided by the user."""
)


In [85]:
print(calculator_agent)
print("#"*50)
calculate_task

formatting_errors=0 id=UUID('7844e866-a171-4dee-aace-51664631a8af') role='mathematical assistant' goal='help users evaluate mathematical expressions accurately' backstory='You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information\n     allows you to provide actionable insights, identify future trends, and assist researchers and professionals in\n     navigating the rapidly evolving landscape of technology.' cache=True config=None max_rpm=None verbose=False allow_delegation=False tools=[CalculateTool(name='Calculate Tool', description="Calculate Tool(expression: 'string') - Evaluates a mathematical expression and returns the result as a float.", args_schema=<class 'abc.CalculateToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x797c91785510>)] max_iter=25 max_execution_time=None agent_executor=CrewAgentExecutor(agent=RunnableAgent(runnable={
  input: RunnableLambda(...),
  tools: RunnableLambda(...),
  too

Task(description=

Question: {query}.


Chain of Thought Example(s):
    
Question: What is the result of 23 * 7 + 15?
Thought: I need to perform a mathematical calculation. I'll use the Calculate Tool for this.
Action: Calculate Tool: (expression: 23 * 7 + 15)
PAUSE
Observation: The result is 176.0
Thought: The calculation is complete. I can now provide the answer.
Answer: The result of 23 * 7 + 15 is 176.



Begin!, expected_output=Answer the mathematical expression provided by the user.)

In [86]:

from crewai import Task
from jinja2 import Template

# currency_converter_persona=for currency conversion
currency_converter_persona = {
    "role": "currency conversion assistant",
    "goal": "help users convert currencies accurately using real-time exchange rates",
    "backstory": """You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to"
provide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology."""
}
currency_converter_agent= Agent(
    llm=llm,
    tools=[currency_converter_tool],  # Ensure tools are passed correctly
    role=currency_converter_persona["role"],
    goal=currency_converter_persona["goal"],  # Pass an actual topic
    backstory=currency_converter_persona["backstory"]  # Add the missing backstory
    , allow_delegation=False, allow_self_execution=True
)


# Example COT for currency conversion task
currency_converter_cot_example = """
Question: What is USD 50 equivalent in EUR?
Thought: I need to convert 50 USD to EUR. I'll use the Currency Converter Tool for this.
Action: Currency Converter Tool: 50, "USD", "EUR"
PAUSE
Observation: 50 USD is equal to 42.15 EUR
Thought: The conversion is complete. I can now provide the answer.
Answer: 50 USD is equivalent to 42.15 EUR.
"""

# Define the task using the Task object
currency_converter_task = Task(
    description=Template(system_prompt_template).render(
        query="{query}",
        cot_example=currency_converter_cot_example
    ),
    agent=currency_converter_agent,
    expected_output="""Answer the user's currency conversion request using real-time exchange rates."""
)


In [87]:
print(currency_converter_agent)
print("#"*50)
currency_converter_task

formatting_errors=0 id=UUID('8bc9822a-8ebe-40d0-9bf4-1d02bfab3659') role='currency conversion assistant' goal='help users convert currencies accurately using real-time exchange rates' backstory='You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to"\nprovide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology.' cache=True config=None max_rpm=None verbose=False allow_delegation=False tools=[CurrencyConverterTool(name='Currency Converter Tool', description="Currency Converter Tool(amount: 'string', from_currency: 'string', to_currency: 'string') - Converts an amount from one currency to another using real-time exchange rates.", args_schema=<class 'abc.CurrencyConverterToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x797c91785510>)] max_iter=25 max_execution_time=None agent_executor=CrewAgentExecu

Task(description=

Question: {query}.


Chain of Thought Example(s):
    
Question: What is USD 50 equivalent in EUR?
Thought: I need to convert 50 USD to EUR. I'll use the Currency Converter Tool for this.
Action: Currency Converter Tool: 50, "USD", "EUR"
PAUSE
Observation: 50 USD is equal to 42.15 EUR
Thought: The conversion is complete. I can now provide the answer.
Answer: 50 USD is equivalent to 42.15 EUR.



Begin!, expected_output=Answer the user's currency conversion request using real-time exchange rates.)

In [88]:



# searcher_persona for web search
researcher_persona = {
    "role": "web search assistant",
    "goal": "help users find information on Serper Search Tool",
    "backstory": "You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to provide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology."
}
researcher_agent= Agent(
    llm=llm,
    tools=[search_tool],  # Ensure tools are passed correctly
    role=researcher_persona["role"],
    goal=researcher_persona["goal"],  # Pass an actual topic
    backstory=researcher_persona["backstory"]  # Add the missing backstory
    , allow_delegation=False, allow_self_execution=True
)


search_cot_example  = """
Thought: I need to find recent trends in {query}. I'll start with a broad search.
Action: Search the internet(search_query:"{query}" )
Observation: (Search results about recent {query} developments, news, and trends)
Thought: I have a comprehensive list now. I'll compile the top 15 trends.
Final Answer: Here's a list of 10 significant {query} related trends,developments...
"""


# Define the task using the Task object
search_task = Task(
    description=Template(system_prompt_template).render(
         query="{query}",
        cot_example=search_cot_example
    ),
    agent=researcher_agent,
    expected_output="""Provide the user with search results based on the query."""
)


In [89]:
print(researcher_agent)
print("#"*50)
search_task

formatting_errors=0 id=UUID('94d982af-6bb1-4cfe-9f7f-e8833a74c31c') role='web search assistant' goal='help users find information on Serper Search Tool' backstory='You are a highly knowledgeable expert assistant agent. Your ability to synthesize complex information allows you to provide actionable insights, identify future trends, and assist researchers and professionals in navigating the rapidly evolving landscape of technology.' cache=True config=None max_rpm=None verbose=False allow_delegation=False tools=[SerperDevTool(name='Search the internet', description="Search the internet(search_query: 'string') - A tool that can be used to semantic search a query from a txt's content.", args_schema=<class 'crewai_tools.tools.serper_dev_tool.serper_dev_tool.SerperDevToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x797c91785510>, search_url='https://google.serper.dev/search', n_results=None)] max_iter=25 max_execution_time=None agent_executor=CrewAgentE

Task(description=

Question: {query}.


Chain of Thought Example(s):
    
Thought: I need to find recent trends in {query}. I'll start with a broad search.
Action: Search the internet(search_query:"{query}" )
Observation: (Search results about recent {query} developments, news, and trends)
Thought: I have a comprehensive list now. I'll compile the top 15 trends.
Final Answer: Here's a list of 10 significant {query} related trends,developments...



Begin!, expected_output=Provide the user with search results based on the query.)

In [90]:
def run_crewai(query:
               str) -> str:
    # Create the crew
    ai_trends_crew = Crew(
        agents=[researcher_agent],
        tasks=[search_task],
        verbose=True,
        process=Process.sequential
    )
    result = ai_trends_crew.kickoff({"query": query})
    return result

if __name__ == "__main__":
    # Run CrewAI for the topic 'AI and Data Science'
    result = run_crewai("What's are the top 10 latest news about AI ?")
    print(result)

 [DEBUG]: == Working Agent: web search assistant
 [INFO]: == Starting Task: 

Question: What's are the top 10 latest news about AI ?.


Chain of Thought Example(s):
    
Thought: I need to find recent trends in What's are the top 10 latest news about AI ?. I'll start with a broad search.
Action: Search the internet(search_query:"What's are the top 10 latest news about AI ?" )
Observation: (Search results about recent What's are the top 10 latest news about AI ? developments, news, and trends)
Thought: I have a comprehensive list now. I'll compile the top 15 trends.
Final Answer: Here's a list of 10 significant What's are the top 10 latest news about AI ? related trends,developments...



Begin!
 


Search results: Title: Artificial Intelligence - Latest AI News and Analysis - WSJ.com
Link: https://www.wsj.com/tech/ai
Snippet: AI Agents Can Do More Than Answer Queries. · Financial Firms Need to Focus on Cyber Risks Posed by AI, New York Regulator Says · Nvidia Will Dominate Big Tech's .

In [28]:
from IPython.display import display, Markdown
display(Markdown(result))

Title: Artificial Intelligence - Latest AI News and Analysis - WSJ.com
Link: https://www.wsj.com/tech/ai
Snippet: AI Agents Can Do More Than Answer Queries. · Financial Firms Need to Focus on Cyber Risks Posed by AI, New York Regulator Says · Nvidia Will Dominate Big Tech's ...

Title: AI News - Artificial Intelligence News
Link: https://www.artificialintelligence-news.com/
Snippet: Latest News · Telefónica's Wayra backs AI answer engine Perplexity · Innovative machine learning uses transforming business applications · UK secures £6.3B in data ...

Title: Artificial intelligence - NBC News
Link: https://www.nbcnews.com/artificial-intelligence
Snippet: The latest news and top stories on artificial intelligence, including AI chatbots like Microsoft's ChatGPT, Apple's AI Chatbot and Google's Bard.

Title: Artificial Intelligence News - ScienceDaily
Link: https://www.sciencedaily.com/news/computers_math/artificial_intelligence/
Snippet: Latest Headlines · Freeing Up Space for Robots to 'Think' · Simulation Mimics How Brain Grows Neurons · Min-Robots in Convoy for Endoscopic Surgery · Helping Robots ...

Title: AI News & Artificial Intelligence - TechCrunch
Link: https://techcrunch.com/category/artificial-intelligence/
Snippet: Penguin Random House is adding an AI warning to its books' copyright pages · Midjourney plans to let anyone on the web edit images with AI · Women in AI: Dr.

Title: Artificial Intelligence - The New York Times
Link: https://www.nytimes.com/spotlight/artificial-intelligence
Snippet: Microsoft and OpenAI's Close Partnership Shows Signs of Fraying. The “best bromance in tech” has had a reality check as OpenAI has tried to change its deal ...

Title: Artificial intelligence (AI) | The Guardian
Link: https://www.theguardian.com/technology/artificialintelligenceai
Snippet: 11 October 2024 · Demis Hassabis and Geoffrey Hinton. 'They don't just fall out of trees': Nobel awards highlight Britain's AI pedigree. Oct 11 2024 08.22 EDT.

Title: AI Magazine: Home of AI and Artificial Intelligence News
Link: https://aimagazine.com/
Snippet: AI Is Ready, Are You? Insights from Camwood MD Andrew Carr · Google's Plan for Energy-Hungry AI: Nuclear Power · Why Nvidia is Helping Bring AI to Container ...

Title: Artificial intelligence - BBC News
Link: https://www.bbc.com/news/topics/ce1qrvleleqt
Snippet: How 1984's The Terminator predicted our AI fears · News app shows fake bloodied Alex Jones AI advert · Museum visitors to have AI chats with dodo.

Title: TOP 25 Artificial Intelligence and Big Data News Publications
Link: https://certainly.io/blog/top-ai-big-data-publications/
Snippet: We have gathered top tech publications about latest big data trends including Artificial Intelligence, Big Data and Machine Learning, Deep Learning and trends ...

In [31]:
def run_crewai(query:
               str) -> str:
    # Create the crew
    ai_trends_crew = Crew(
        agents=[calculator_agent],
        tasks=[calculate_task],
        verbose=True,
        process=Process.sequential,
        full_output=True,
    )
    result = ai_trends_crew.kickoff({"query": query})
    return result

if __name__ == "__main__":

    result = run_crewai("what is 10*25-25")

 [DEBUG]: == Working Agent: mathematical assistant
 [INFO]: == Starting Task: 

Question: what is 10*25-25.


Chain of Thought Example(s):
    
Question: What is the result of 23 * 7 + 15?
Thought: I need to perform a mathematical calculation. I'll use the Calculate Tool for this.
Action: Calculate Tool: (expression: 23 * 7 + 15)
PAUSE
Observation: The result is 176.0
Thought: The calculation is complete. I can now provide the answer.
Answer: The result of 23 * 7 + 15 is 176.



Begin!
-> TOOL-CALCULATE CALLED
 

225.0

 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



 

I tried reusing the same input, I must 

In [32]:
def run_crewai(query:
               str) -> str:
    # Create the crew
    ai_trends_crew = Crew(
        agents=[currency_converter_agent],
        tasks=[currency_converter_task],
        verbose=True,
        process=Process.sequential,
        full_output=True,
    )
    result = ai_trends_crew.kickoff({"query": query})
    return result

if __name__ == "__main__":

    result = run_crewai("What is USD 29 equivalent in GBP")

 [DEBUG]: == Working Agent: currency conversion assistant
 [INFO]: == Starting Task: 

Question: What is USD 29 equivalent in GBP.


Chain of Thought Example(s):
    
Question: What is USD 50 equivalent in EUR?
Thought: I need to convert 50 USD to EUR. I'll use the Currency Converter Tool for this.
Action: Currency Converter Tool: 50, "USD", "EUR"
PAUSE
Observation: 50 USD is equal to 42.15 EUR
Thought: The conversion is complete. I can now provide the answer.
Answer: 50 USD is equivalent to 42.15 EUR.



Begin!
 

22.23

 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



 

I tried reusing the same input, I mus

In [95]:
manager_agent = Agent(
    role="Manager",
    goal="Manage and delegate tasks to the appropriate agents in the crew.",
    backstory=(
        "You are a Manager Agent responsible for coordinating tasks among the agents in the AI trends crew."
        "You understand which agent is best suited for which task and route the tasks accordingly."

    ),
    verbose=True,
    llm=llm,  # You can use the Groq LLM or any other completion function here
    allow_delegation=True,
    allow_self_execution=True,

)

In [96]:
def run_crewai(query:
               str) -> str:
    # Create the crew
    ai_trends_crew = Crew(
        agents=[calculator_agent,currency_converter_agent, researcher_agent],
        tasks=[calculate_task,currency_converter_task, search_task],
        verbose=True,
        process=Process.hierarchical,
        manager_agent=manager_agent,
        manager_llm=llm,
        full_output=True,

    )
    result = ai_trends_crew.kickoff({"query": query})
    return result

if __name__ == "__main__":
    # Run CrewAI for the topic 'AI and Data Science'
    result = run_crewai("What's are the top 10 latest news about AI ?")
    print(result)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: 

Question: What's are the top 10 latest news about AI ?.


Chain of Thought Example(s):
    
Question: What is the result of 23 * 7 + 15?
Thought: I need to perform a mathematical calculation. I'll use the Calculate Tool for this.
Action: Calculate Tool: (expression: 23 * 7 + 15)
PAUSE
Observation: The result is 176.0
Thought: The calculation is complete. I can now provide the answer.
Answer: The result of 23 * 7 + 15 is 176.



Begin!


> Entering new CrewAgentExecutor chain...
Thought: I need to find the top 10 latest news about AI. This requires searching the web for relevant news articles. I'll delegate this task to the web search assistant.

Action: Delegate work to co-worker
Action Input: {"coworker": "web search assistant", "task": "find the top 10 latest news about AI", "context": "search the web for news articles related to AI, prioritize recent articles from reputable sources"}

PAUSE
I apologize for the mistake. 

KeyboardInterrupt: 

In [93]:
def run_crewai(query:
               str) -> str:
    # Create the crew
    ai_trends_crew = Crew(
        agents=[calculator_agent,currency_converter_agent, researcher_agent],
        tasks=[calculate_task,currency_converter_task, search_task],
        verbose=True,
        process=Process.hierarchical,
        manager_agent=manager_agent,
        manager_llm=llm,
        full_output=True,
    )
    result = ai_trends_crew.kickoff({"query": query})
    return result

if __name__ == "__main__":
    # Run CrewAI for the topic 'AI and Data Science'
    result = run_crewai("What's are the top 10 latest news about AI ? and 100 usd ie equal to how many inr?")
    print(result)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: 

Question: What's are the top 10 latest news about AI ? and 100 usd ie equal to how many inr?.


Chain of Thought Example(s):
    
Question: What is the result of 23 * 7 + 15?
Thought: I need to perform a mathematical calculation. I'll use the Calculate Tool for this.
Action: Calculate Tool: (expression: 23 * 7 + 15)
PAUSE
Observation: The result is 176.0
Thought: The calculation is complete. I can now provide the answer.
Answer: The result of 23 * 7 + 15 is 176.



Begin!


> Entering new CrewAgentExecutor chain...
Thought: I need to break down the task into smaller sub-tasks to manage the team effectively.

Action: Delegate work to co-worker
Action Input: {"coworker": "web search assistant", "task": "find the top 10 latest news about AI", "context": "Please search the internet and provide the top 10 latest news articles about Artificial Intelligence. Make sure the articles are from reputable sources and are recent."}

(PA

KeyboardInterrupt: 